# From Primary Source to Data
This Jupyter Notebook is a step-by-step tutorial of the text extraction and dataset building process for the primary source: The Letters of Lady Montagu, originally published in 1763. 


## Text Extraction

This tutorial uses an epub copy of the 1790 edition of Montagu's letters. Instead of dealing with low-quality OCR output and complications related to 18th century orthography and spelling, we chose to use a machine readable copy.

In [ ]:
!pip install ebooklib

In [ ]:
## Step 1: Text Extraction from ePub

import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup

def extract_text_from_epub(epub_path, output_txt_path):
    book = epub.read_epub(epub_path)
    text_content = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_content(), 'html.parser')
            text_content.append(soup.get_text())

    # Join extracted text and save to a file
    with open(output_txt_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(text_content))

    print(f"Text extracted and saved to {output_txt_path}")

# Example usage
epub_file = "../Data/montagu/montagu_2019.epub" 
output_text_file = "../Data/montagu/montagu_2019_unedited.txt"
extract_text_from_epub(epub_file, output_text_file)

### Glitches

Historical data is not always very clean, even if it is already processed by contemporary scholars. In this example, I know that I only wanted the letters but there are introductory materials and other scholarly commentary. I prefer to edit these out and often it is easier to do it manually.

After this step, I created a copy of montagu_2019_unedited and renamed it to montagu_2019. Then, I manually removed the materials before the letters and the materials after the letters and saved them to different text files (not shared here). This made the text segmentation process below easier.

In [ ]:
## Step 2: Splitting Letters

import os
import re

def split_letters(file_path, output_folder):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Read the entire text file
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    # Use regex to find all letters (assuming they start with "LETTER. X")
    # Improved regex to handle "LETTER VIII", "LET. VIII", and other variations
    letters = re.split(r"(?=\nLET(?:TER)?(?:\.|\s)\s*[IVXLCDM\d]+)", text)

    # Remove any empty splits
    letters = [letter.strip() for letter in letters if letter.strip()]

    # Save each letter as a separate file
    for i, letter in enumerate(letters, start=1):
        file_name = f"letter_{i}.txt"
        with open(os.path.join(output_folder, file_name), "w", encoding="utf-8") as letter_file:
            letter_file.write(letter)

    print(f"Split {len(letters)} letters into individual files in '{output_folder}'.")

# Example usage
input_file = "../Data/montagu/montagu_2019.txt"  # Path to uploaded file
output_directory = "../Data/montagu/montagu_letters"  # Folder for individual letter files
split_letters(input_file, output_directory)

### Why does this work?

In the code below we relied on a few assumptions about the structure of the extracted letters.

Firstly, We know that the all the extracted letters begin with the letter title. So, we could savely assume this: title = lines[0]

Secondly, we know that the addressee starts with a version of "to" (TO; To; to). With this in mind, we created a regex that searches for this.

Thirdly, we know that the location and date information are in this format: "location, date" (ie: Nuremberg, Aug. 22. O. S. 1716.). We also know that location,date is often the first line with text after the addressee line. We used this information to simplify our search in finding and splitting the location,date. 

At the same time we know that some letters have an editor's note between the addressee and the location,date column in this format: [Footnote: ....] Thus, we introduced a skipping condition to skip the line that contains 'Footnote:'

Lastly, we designated everything after location,date col to be the body of the letter.

Based on these assumptions, we ended up with 51 letters correctly extracted and only 7 needing manual editing. 

In [ ]:
# Step 3: CSV
"""
Our goal is to create a CSV with the following columns:
filename, title, addressee, location, date, body

For example
letter_5.txt, LET. V., THE COUNTESS OF B——., Nuremberg, Aug. 22. O. S. 1716., textoftheletter 
"""
import csv

# Define the folder where the letters are stored
letters_folder = "../Data/montagu/montagu_letters" 
output_file = "../Data/montagu/montagu_letters_unedited.csv"

# Helper function to extract the numeric index from the filename
def get_letter_number(filename):
    # Matches filenames like "letter_1.txt" -> returns integer 1
    match = re.match(r"letter_(\d+)\.txt", filename)
    if match:
        return int(match.group(1))
    return float("inf")  # Fallback if it doesn't match the pattern

# List all text files in the folder, then sort numerically
letter_files = [f for f in os.listdir(letters_folder) if f.startswith("letter_") and f.endswith(".txt")]
letter_files.sort(key=get_letter_number)

# Prepare the CSV headers
headers = ["filename", "title", "addressee", "location", "date", "body"]

# Initialize storage for letters
letters_data = []

# Loop through each letter file
for letter_file in letter_files:
    file_path = os.path.join(letters_folder, letter_file)
    print(f"Processing: {file_path}")

    # Read the content of the letter
    with open(file_path, "r", encoding="utf-8") as file:
        lines = [line.strip() for line in file.readlines() if line.strip()]

    if not lines:
        continue  # Skip empty files

    # Extract filename
    filename = letter_file

    # Extract title (first line)
    title = lines[0]

    # Extract addressee (line starting with "To ", "to ", or "TO ")
    addressee = "Unknown"
    addressee_index = None
    for idx, line in enumerate(lines):
        if re.match(r"^(To|TO|to)\s+(.*)", line):
            addressee = line.strip()
            addressee_index = idx
            break

    # Extract location and date (first line after addressee, ignoring footnotes)
    location, date = "Unknown", "Unknown"
    location_date_line = None

    if addressee_index is not None:
        for line in lines[addressee_index + 1:]:  # start after the addressee line
            if "Footnote:" in line:
                continue  # skip footnotes
            location_date_line = line
            break

    if location_date_line:
        # Split by the first comma
        parts = location_date_line.split(",", 1)
        if len(parts) == 2:
            location = parts[0].strip()
            date = parts[1].strip()

    # Extract body (everything after location/date line)
    if location_date_line and location_date_line in lines:
        body_start_idx = lines.index(location_date_line) + 1
    else:
        body_start_idx = addressee_index + 1 if addressee_index is not None else 1

    body = "\n".join(lines[body_start_idx:]).strip()

    # Store data
    letters_data.append([filename, title, addressee, location, date, body])

# Write data to CSV file
with open(output_file, "w", encoding="utf-8", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)  # Write header row
    writer.writerows(letters_data)  # Write letter rows

print(f"CSV file '{output_file}' has been created successfully.")


In [1]:
import pandas as pd

# Load the CSV file into a DataFrame
montagu_unedited = pd.read_csv('../Data/montagu/montagu_letters_unedited.csv')
montagu_unedited.head()

,filename,title,addressee,location,date,body
0,letter_1.txt,LETTER I.,TO THE COUNTESS OF ——.,Rotterdam,Aug. 3. O. S. 1716.,"I FLATTER, myself, dear sister, that I shall g..."
1,letter_2.txt,LET. II,TO MRS. S——.,Hague,Aug. 5. O. S. 1716.,"I MAKE haste to tell you, dear Madam, that, af..."
2,letter_3.txt,LET. III,TO MRS. S. C.,Nimeguen,Aug.13. O. S. 1716.,"I AM extremely sorry, my dear S. that your fea..."
3,letter_4.txt,LET. IV.,TO THE LADY ——.,Cologn (sic),"Aug, 16. O. S. 1716.",IF my lady —— could have any notion of the fat...
4,letter_5.txt,LET. V.,TO THE COUNTESS OF B——.,Nuremberg,Aug. 22. O. S. 1716.,"AFTER five days travelling post, I could not s..."


In [ ]:
# Let's make sure that we got all the 58 letters
montagu_unedited.shape

In [ ]:
# Some descriptive statistics, especially relevant are unique counts of addressee, location, and date
montagu_unedited.describe()

In [ ]:
# What are the data types of the columns?
print(montagu_unedited.dtypes)

In [ ]:
# Check for missing values in the DataFrame
print(montagu_unedited.isnull().sum())

In [ ]:
# Another way to check for missing values
montagu_unedited.isna().sum()

In [ ]:
# Compare isna() vs isna().sum()
# montagu_unedited.isna() returns a DataFrame of boolean values indicating missing data
# montagu_unedited.isna().sum() returns a Series with the count of missing values per column

montagu_unedited.isna()

# This has its own uses too but clearly not what we want here

In [ ]:
# We could have also done this with the `info()` method
montagu_unedited.info()

In [ ]:
# This seems fine?? Are we ready to move on?

# Let's see if we have any unkowns
print('addressee')
print(montagu_unedited[montagu_unedited['addressee'] == 'Unknown'])

print('location')
print(montagu_unedited[montagu_unedited['location'] == 'Unknown'])

print('date')
print(montagu_unedited[montagu_unedited['date'] == 'Unknown'])

### Unknown

We employed this trick in our code, where we first created everything with the string 'Unknown' and then filled it in if the information was there. Here is the snippet:

    addressee = "Unknown"
    addressee_index = None
    for idx, line in enumerate(lines):
        if re.match(r"^(To|TO|to)\s+(.*)", line):
            addressee = line.strip()
            addressee_index = idx
            break


In [ ]:
# Let's get all the unknowns and see how many we have and in which rows
unknowns = montagu_unedited[(montagu_unedited['addressee'] == 'Unknown') |
                            (montagu_unedited['location'] == 'Unknown') |
                            (montagu_unedited['date'] == 'Unknown')]
print(unknowns)

### Editing

At this point, I decided to edit these unknowns manually. While we want to do a lot of things with code, we also shouldn't forget that manual data cleaning is always an option. Besides you will learn a lot about your dataset by doing that.

I edited these issues and continued with data augmentation on Google Sheets.

Oh and before I forget, I also edited the **dates**. the standard format for dates in computational research is: YYYY-MM-DD

## Data Augmentation

There are three directions that we can go from here

1. **Location** -> Remember the example from '2-4_pandas.ipynb' and how it made use of the locations that the letters were sent to and from? We can do that here too!
    
    For this particular task, I prefer using Google Sheets, especially since we have only a handful letters. My goal here is to ensure that I have a clean csv where the location is uniformly edited and that I can use this location information to augment my dataset.

    This augmentation process includes adding Wikidata IDs and coordinates.

    The result of this process can be found in "Data/montagu/montagu_letters_for_mapping.csv"

2. **Letter bodies** -> We can start analyzing the actual texts of the letters
    This task includes normalizing the texts, running algorithms like POS-tagging and Named Entity Recognition and even getting embeddings for our letter.

3. **People** -> In this edited volume that we extracted the letters from, the editors were kept everything the same as the original 1790 edition. However, in some other editions of Montagu's letters, different scholars have chosen to identify who the recipients were. We can look this information up to augment our dataset

### Letter bodies

Let's start looking at the letter bodies and what they contain, similar to how we analyzed Borges' poem in 2-6_text_analysis_introduction.ipynb



In [2]:
# Let's get the most updated version of our data
# We will use the version that has been manually corrected

montagu = pd.read_csv('../Data/montagu/montagu_letters_v1.csv')

In [ ]:
montagu.head()

In [ ]:
# Looks good! Let's get one letter and start working with it

letter_35 = montagu['body'].iloc[34]  # Remember that iloc is zero-indexed
print(letter_35)


In [ ]:
# Let's get some basic statistics about the letter

print(f"Length of letter 35: {len(letter_35)} characters")
print(f"Number of words in letter 35: {len(letter_35.split())} words")

In [ ]:
from string import punctuation

punct_count = 0
for char in punctuation:
    punct_count += letter_35.count(char)
print(punct_count)

In [ ]:
sum = 0
print(sum)

In [ ]:
sum += 1
print(sum)

In [ ]:
from string import punctuation
print(punctuation)
print(len(punctuation))

In [ ]:
# How many punctuation marks are in the letter?

punctuation_count = 0

for char in letter_35:
    if char in punctuation:
        punctuation_count += 1

print(punctuation_count)

In [ ]:
print("Percentage of punctuation marks in letter 35:")
print(f"{(punctuation_count / len(letter_35)) * 100:.2f}%")

In [ ]:
# Before we continue, let's split the letter into sentences

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

# Tokenize the letter into sentences
letter_35_sentences = sent_tokenize(letter_35)

print(f"Number of sentences in letter 35: {len(letter_35_sentences)}")

counter = 1
for sentence in letter_35_sentences:
    print(counter) 
    print(sentence)
    counter += 1

In [ ]:
# When we split it sentence by sentence, we can see that the sentences have some line break issues
# Sometimes analyzing the text can reveal formatting issues that need to be addressed, 
# especially when dealing with historical texts or texts extracted from formats like ePub.

# Let's fix the line breaks in the actual letter

letter_35_test = letter_35.replace('\n', ' ')

letter_35_sentences_test = sent_tokenize(letter_35_test)

print(f"Number of sentences in letter 35: {len(letter_35_sentences_test)}")

counter = 1
for sentence in letter_35_sentences_test:
    print(counter) 
    print(sentence)
    counter += 1


In [ ]:
# Now we can see that the sentences are much cleaner and easier to read
# Let's save the cleaned letter back to the DataFrame and use the same approach for all letters

# We will talk more about the lambda function next week

montagu['body_cleaned'] = montagu['body'].apply(lambda x: x.replace('\n', ' '))

# Now let's save the updated DataFrame to a new CSV file
montagu.to_csv('../Data/montagu/montagu_letters_v2.csv', index=False) # note the name change here! versioning is important

In [3]:
# Let's go back to working with Letter 35
# We will override the previous letter_35 variable to use the cleaned version

import pandas as pd
montagu = pd.read_csv('../Data/montagu/montagu_letters_v2.csv')

letter_35 = montagu['body_cleaned'].iloc[34]  # Remember that iloc is zero-indexed

In [4]:
print(letter_35)

I HAVE had the advantage of very fine weather, all my journey; and as the summer is now in its beauty, I enjoyed the pleasure of fine prospects; and the meadows being full of all sorts of garden flowers, and sweet herbs, my berlin perfumed the air as it pressed them. The grand signior furnished us with thirty covered waggons for our baggage, and five coaches of the country for my women. We found the road full of the great spahis and their equipages coming out of Asia to the war. They always travel with tents; but I chose to ly in houses all the way. I will not trouble you with the names of the villages we passed, in which there was nothing remarkable, but at Ciorlei, where there was a conac, or little seraglio, built for the use of the grand signior, when he goes this road. I had the curiosity to view all the apartments destined for the ladies of his court. They were in the midst of a thick grove of trees, made fresh by fountains; but I was most surprised to see the walls almost covere

In [ ]:
print(f"Length of letter 35: {len(letter_35)} characters")
print(f"Number of words in letter 35: {len(letter_35.split())} words")

In [ ]:
# How many unique words are in the letter?

print(f"Unique words in letter 35: {len(set(letter_35.split()))}")

In [ ]:
# Aren't we forgetting something?

# Ah yes, lowercase! Let's convert the letter to lowercase and then count unique words

letter_35_lower = letter_35.lower()
unique_words = set(letter_35_lower.split())
print(f"Unique words in letter 35 (lowercase): {len(unique_words)}")

In [ ]:
# Let's tokenize our sentences again and this time we will use the lowercase version of the letter
sentences = sent_tokenize(letter_35_lower)

counter = 1
for sentence in sentences[:5]: # only print the first 5
    print(counter) 
    print(sentence)
    counter += 1

In [ ]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger_eng') 
from nltk.tokenize import word_tokenize

# Sample sentence for POS tagging
sample_sentence = sentences[0]  # First sentence for demonstration

# Apply POS tagging
pos_tags = pos_tag(word_tokenize(sample_sentence))

print(f"POS Tags for the sample sentence: {pos_tags}")


### What are POS tags anyways?

POS tagging (Part-of-Speech tagging) is the process of assigning a specific part-of-speech category (e.g., noun, verb, adjective, etc.) to each word in a sentence. This helps in understanding the grammatical structure of a sentence and is a key step in many natural language processing (NLP) tasks such as information extraction, machine translation, and text analysis.

POS tags typically include categories like:

- Nouns (e.g., "dog", "city")

- Verbs (e.g., "run", "is")

- Adjectives (e.g., "quick", "beautiful")

- Adverbs (e.g., "quickly", "very")

- Pronouns (e.g., "he", "they")

- Prepositions (e.g., "in", "on")

- Conjunctions (e.g., "and", "but")





### POS Tags in our example

- **NN (Noun, Singular)**: Represents singular nouns.
    - *advantage, weather, journey, summer, beauty, pleasure, prospects, meadows, flowers, herbs, air*

- **VBP (Verb, Non-3rd Person Singular Present)**: A verb in the present tense used with subjects other than third-person singular (he/she/it).
    - *have*

- **VBN (Verb, Past Participle)**: A verb in past participle form, often used with auxiliary verbs.
    - *had*

- **DT (Determiner)**: A word that introduces a noun.
    - *the, all*

- **IN (Preposition/Subordinating Conjunction)**: Used to show relationships between elements in the sentence.
    - *of, in, as*

- **RB (Adverb)**: Modifies or describes a verb, adjective, or another adverb.
    - *very, now*

- **JJ (Adjective)**: Describes or modifies a noun.
    - *fine, sweet*

- **PRP$ (Possessive Pronoun)**: A pronoun that shows ownership or possession.
    - *my, its*

- **VBD (Verb, Past Tense)**: A verb in the simple past tense.
    - *enjoyed, perfumed, pressed*

- **CC (Coordinating Conjunction)**: Used to connect words, phrases, or clauses of equal importance.
    - *and*

- **PRP (Personal Pronoun)**: Refers to a person or thing.
    - *I, it, them*

- **NNS (Noun, Plural)**: Represents plural nouns.
    - *prospects, meadows, sorts, flowers, herbs*

- **. (Punctuation)**: Marks the end of a sentence or clause.
    - *.*


### What can we do with POS Tagging?

We can analyze the structure of the sentences, identify parts of speech, and even extract specific types of words.

For example, we can extract all adjectives from this letter.

In [ ]:
# What can we do with POS Tagging?
# We can analyze the structure of the sentences, identify parts of speech, and even extract specific types of words.
# For example, we can extract all adjectives from this letter.

# First we need to create a function that extracts adjectives from a sentence
def extract_adjectives(sentence):
    words = word_tokenize(sentence)
    pos_tags = pos_tag(words)
    adjectives = [word for word, tag in pos_tags if tag.startswith('JJ')]  # 'JJ' is the tag for adjectives
    return adjectives



In [ ]:
# Let's check to see that our function works
sample_sentence = sentences[0]  # First sentence for demonstration
adjectives = extract_adjectives(sample_sentence)
print(f"Adjectives in the sample sentence: {adjectives}")

In [ ]:
# Our function works! But it is not without issues.
# Why is i and full in the list of adjectives?

print(sample_sentence)
print(f"POS Tags for the sample sentence: {pos_tags}")

POS Tags for the sample sentence: [('i', 'NN'), ('have', 'VBP'), ('had', 'VBN'), ('the', 'DT'), ('advantage', 'NN'), ('of', 'IN'), ('very', 'RB'), **('fine', 'JJ')**, ('weather', 'NN'), (',', ','), ('all', 'DT'), ('my', 'PRP$'), ('journey', 'NN'), (';', ':'), ('and', 'CC'), ('as', 'IN'), ('the', 'DT'), ('summer', 'NN'), ('is', 'VBZ'), ('now', 'RB'), ('in', 'IN'), ('its', 'PRP$'), ('beauty', 'NN'), (',', ','), **('i', 'JJ')**, ('enjoyed', 'VBD'), ('the', 'DT'), ('pleasure', 'NN'), ('of', 'IN'), **('fine', 'JJ')**, ('prospects', 'NNS'), (';', ':'), ('and', 'CC'), ('the', 'DT'), ('meadows', 'NNS'), ('being', 'VBG'), **('full', 'JJ')**, ('of', 'IN'), ('all', 'DT'), ('sorts', 'NNS'), ('of', 'IN'), ('garden', 'NN'), ('flowers', 'NNS'), (',', ','), ('and', 'CC'), **('sweet', 'JJ')**, ('herbs', 'NN'), (',', ','), ('my', 'PRP$'), ('berlin', 'NN'), ('perfumed', 'VBD'), ('the', 'DT'), ('air', 'NN'), ('as', 'IN'), ('it', 'PRP'), ('pressed', 'VBD'), ('them', 'PRP'), ('.', '.')]

In [ ]:
# We can improve our function by filtering out certain words that are not adjectives

# Here we rely on a simple heuristic: if the word that is tagged as an adjective is followed by a noun, it is likely an adjective.

def extract_adjectives_improved(sentence):
    sentence = sentence.lower()  # Convert to lowercase for consistency
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    adjectives = []
    for i in range(len(tagged_words) - 1):
        current_word, current_tag = tagged_words[i]
        next_word, next_tag = tagged_words[i + 1]
        if current_tag.startswith('JJ') and next_tag.startswith('NN'):
            adjectives.append(current_word)
    return adjectives

In [ ]:
print(extract_adjectives_improved(sample_sentence))

Much better! Let's scale this up!

In [ ]:
# Let's get adjectives from all sentences in the letter
all_adjectives = []
for sentence in sentences:
    adjectives = extract_adjectives_improved(sentence)
    all_adjectives.extend(adjectives)

print(all_adjectives)

### Why extend and not append?

Last week we learnt to use `append()` to add an element to a list. Now we are using `extend()` instead. Why?

`append()` simply adds 'adjectives' in each iteration of the for loop to the list 'all_adjectives'. In this case 'adjectives' is also a list. This means that if we used `append()` we would be creating a list of lists. What we actually want is to create a list of strings, so we need to `extend()` our list 'all_adjectives' with the strings in the list 'adjectives' every time that list is created. 

In [ ]:
# Let's get the unique adjectives from the letter and count them
print(f"Unique adjectives in letter 35: {len(set(all_adjectives))}")

In [ ]:
# Let's see all the unique adjectives in the letter and their counts
from collections import Counter
adjective_counts = Counter(all_adjectives)

# Print the most common 10 adjectives
for adj in adjective_counts.most_common(10):
    print(f"{adj[0]}: {adj[1]}")


In [ ]:
# Perfect! Now let's do this for all letters in the dataset and save the results in a new column in the DataFrame
montagu['adjectives'] = montagu['body_cleaned'].apply(lambda x: extract_adjectives_improved(x))

In [ ]:
montagu['adjectives'].head()

In [ ]:
# Finally, let's get all the unique adjectives from all letters and their counts

adjectives_all_letters = []
for adjectives in montagu['adjectives']:
    adjectives_all_letters.extend(adjectives)

adjective_counts_all_letters = Counter(adjectives_all_letters)

for adj in adjective_counts_all_letters.most_common(10):
    print(f"{adj[0]}: {adj[1]}")



In [ ]:
# Let's visualize the most common adjectives in a bar chart
import matplotlib.pyplot as plt
# Get the most common adjectives and their counts
most_common_adjectives = adjective_counts_all_letters.most_common(10)
adjectives, counts = zip(*most_common_adjectives)
plt.figure(figsize=(12, 6))
plt.bar(adjectives, counts, color='skyblue')
plt.xlabel('Adjectives')
plt.ylabel('Count')
plt.title('Most Common Adjectives in Montagu Letters')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()